<a href="https://colab.research.google.com/github/ctrivino1/ML_class_final/blob/main/Chris_Trivino_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [1]:
!pip install pyforest
!pip install git+https://github.com/keras-team/keras-preprocessing.git
!pip install keras-rectified-adam
!pip install tensorflow_addons
import tensorflow_addons as tfa
from keras_radam import RAdam
import pyforest
import shutil, os
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras_radam import RAdam


  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14607 sha256=6d600aafed0f668ae6aa5e617e7682689f04446d241cf19f0de1648d6bbbabdc
  Stored in directory: /root/.cache/pip/wheels/61/1c/da/48e6c884142d485475d852d69d20a096aba5beceb338822893
Successfully built pyforest
  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-l3ohzhm1
  Running command git clone -q https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-l3ohzhm1
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.19.0-py3-none-any.whl size=9017 sha256=195da2c56a9c8b99d28381c239cbf4b47ed93758c17beaeb7d1c1e2e78e4fc5c
  Stored in directory: /root/.cache/pip/wheels/52/1d/66/374c85b327ea412af5301b4811088408d9764314d6cfb8f127
Successfully built keras-rectified-adam
     |████████████████████████████████| 1.1 MB 4.1 MB/s 


In [2]:
!git clone https://github.com/ctrivino1/ML_class_final.git


Cloning into 'ML_class_final'...
remote: Enumerating objects: 21431, done.
remote: Counting objects: 100% (4311/4311), done.
remote: Compressing objects: 100% (4296/4296), done.
remote: Total 21431 (delta 13), reused 4310 (delta 12), pack-reused 17120
Receiving objects: 100% (21431/21431), 2.38 GiB | 13.80 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Checking out files: 100% (21401/21401), done.


In [3]:
training_dir = "/content/ML_class_final/train_imgs_1"
test_dir = "/content/ML_class_final/test_imgs_1"
df=pd.read_csv("/content/ML_class_final/train_1.csv")
df.label = df.label.astype(str)

<IPython.core.display.Javascript object>

In [4]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=.2,zoom_range=.2,horizontal_flip=True,width_shift_range=0.05,height_shift_range=0.05,validation_split=.2,)


train_generator = train_datagen.flow_from_dataframe(dataframe=df, 
                                         directory=training_dir, 
                                         x_col="image_id", 
                                         y_col="label", 
                                         class_mode="categorical", 
                                         target_size=(256,256), 
                                         batch_size=64,
                                         subset="training",
                                         seed=69,
                                         )






Found 13694 validated image filenames belonging to 5 classes.


In [5]:
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
valid_generator = val_datagen.flow_from_dataframe(dataframe=df, 
                                         directory=training_dir, 
                                         x_col="image_id", 
                                         y_col="label", 
                                         class_mode="categorical", 
                                         target_size=(256,256), 
                                         batch_size=64,
                                         subset="validation",
                                         seed=69)

Found 3423 validated image filenames belonging to 5 classes.


In [6]:

resnet50 = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(256,256,3), pooling="max"
)


vgg16 = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(256,256,3), pooling="max"
)
# Freeze the pretrained CNN's
resnet50.trainable = False
vgg16.trainable = False

# optimizers
radam = tfa.optimizers.RectifiedAdam(learning_rate=1e-3)

# Adds ResNet50 into a model with a output layers with 5 classifaction nodes
model = keras.Sequential([vgg16,
    tf.keras.layers.Dense(units=256, activation='LeakyReLU'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='LeakyReLU'),
    tf.keras.layers.Dense(5,activation="softmax") # number of perceptrons
])
model.compile(loss="categorical_crossentropy",
optimizer=radam, metrics=['acc'])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit(train_generator,
          validation_data = valid_generator,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_steps = STEP_SIZE_VALID,
          epochs=15)





58900480/58889256 [==============================] - 0s 0us/step
Epoch 1/15
213/213 [==============================] - 290s 1s/step - loss: 1.0168 - acc: 0.6408 - val_loss: 0.9361 - val_acc: 0.6557
Epoch 2/15
213/213 [==============================] - 270s 1s/step - loss: 0.8906 - acc: 0.6738 - val_loss: 0.8912 - val_acc: 0.6751
Epoch 3/15
213/213 [==============================] - 274s 1s/step - loss: 0.8619 - acc: 0.6806 - val_loss: 0.8341 - val_acc: 0.6860
Epoch 4/15
213/213 [==============================] - 270s 1s/step - loss: 0.8452 - acc: 0.6895 - val_loss: 0.8482 - val_acc: 0.6987
Epoch 5/15
213/213 [==============================] - 269s 1s/step - loss: 0.8357 - acc: 0.6912 - val_loss: 0.9349 - val_acc: 0.6486
Epoch 6/15
213/213 [==============================] - 269s 1s/step - loss: 0.8316 - acc: 0.6931 - val_loss: 0.8350 - val_acc: 0.6931
Epoch 7/15
213/213 [==============================] - 272s 1s/step - loss: 0.8110 - acc: 0.6982 - val_loss: 0.8056 - val_acc: 0.6999
Epoc

In [7]:
#downloading the model as a zipfile

import shutil
model.save('70_perc_model')
shutil.make_archive('70_perc_model', 'zip', '70_perc_model')

INFO:tensorflow:Assets written to: 70_perc_model/assets


INFO:tensorflow:Assets written to: 70_perc_model/assets


'/content/70_perc_model.zip'

In [9]:
import pathlib


def predict_an_image(model, file_path):
  # Load the image
  img = keras.preprocessing.image.load_img(file_path, target_size=(256, 256))
  # Get the image into the shape we need for our network
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) 
  # Predict the class
  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  return np.argmax(score)
# Loop through all images in our test directory and make
# a prediction
testdir = pathlib.Path('/content/ML_class_final/test_imgs_1')
image_paths = (testdir.glob('*.jpg'))
ip = []
pred = []
for image_path in image_paths:
  prediction = predict_an_image(model, str(image_path))
  ip.append(image_path)
  pred.append(prediction)
  #print(image_path, prediction)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
im_path = np.array(ip)
preds = np.array(pred)
df = pd.DataFrame({'Filename':im_path, 'class':preds})
df.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Filename,class
0,/content/ML_class_final/test_imgs_1/3546777867...,1
1,/content/ML_class_final/test_imgs_1/608792631.jpg,2
2,/content/ML_class_final/test_imgs_1/1154243459...,4
3,/content/ML_class_final/test_imgs_1/387772759.jpg,2
4,/content/ML_class_final/test_imgs_1/600610735.jpg,4


In [12]:
## save the new dataframe to csv
from google.colab import files

df.to_csv('predictions.csv')
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>